# Домашнее задание 6: классификация текстов

В этом домашнем задании вам предстоит построить классификатор текстов!

Данные мы будем использовать из Kaggle соревнования: https://www.kaggle.com/competitions/nlp-getting-started/data Оттуда надо скачать файл train.csv. На обучающую и тестовую выборки его поделим кодом ниже, менять его не надо!

Мы будем работать с датасетом постов из твиттера. Нам предстоит решать задачу бинарной классификации - определять содержатся ли в твитте информация о настоящей катастрофе/инциденте или нет.

In [121]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer


import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
data = pd.read_csv('C:/Users/user/Downloads/train.csv')

In [3]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Задание 1 (0.5 балла)

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их пустой строкой.

In [4]:
data.isnull().any() # пропуски в keyword и location

id          False
keyword      True
location     True
text        False
target      False
dtype: bool

In [5]:
data = data.fillna('')
data.isnull().any() # теперь нету

id          False
keyword     False
location    False
text        False
target      False
dtype: bool

## Задание 2 (1 балл)
Давайте немного посмотрим на наши данные. Визуализируйте (где явно просят) или выведете информацию о следующем:

1. Какое распределение классов в обучающей выборке?
2. Посмотрите на колонку "keyword" - возьмите 10 наиболее встречающихся значений, постройте ступенчатую диаграмму распределения классов в зависимости от значения keyword, сделайте выводы.

In [6]:
data['target'].value_counts() # небольшой перевес класса 0, явного дисбаланса нет

0    4342
1    3271
Name: target, dtype: int64

In [7]:
data['keyword'].value_counts()[:11]

               61
fatalities     45
deluge         42
armageddon     42
sinking        41
damage         41
harm           41
body%20bags    41
evacuate       40
fear           40
outbreak       40
Name: keyword, dtype: int64

## Задание 3 (0.5 балла) 

В этом задании предлагается объединить все три текстовых столбца в один (просто сконкатенировать cтроки) и убрать столбец с индексом.

In [8]:
data = data.drop(labels=['id'], axis=1)
data

,keyword,location,text,target
0,,,Our Deeds are the Reason of this #earthquake M...,1
1,,,Forest fire near La Ronge Sask. Canada,1
2,,,All residents asked to 'shelter in place' are ...,1
3,,,"13,000 people receive #wildfires evacuation or...",1
4,,,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
7608,,,Two giant cranes holding a bridge collapse int...,1
7609,,,@aria_ahrary @TheTawniest The out of control w...,1
7610,,,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,,,Police investigating after an e-bike collided ...,1


In [9]:
data['new'] = data['keyword'] + ' ' + data['location'] +  ' ' + data['text']
data = data.drop(labels = ['keyword', 'location', 'text'], axis = 1)
data

,target,new
0,1,Our Deeds are the Reason of this #earthquake...
1,1,Forest fire near La Ronge Sask. Canada
2,1,All residents asked to 'shelter in place' ar...
3,1,"13,000 people receive #wildfires evacuation ..."
4,1,Just got sent this photo from Ruby #Alaska a...
...,...,...
7608,1,Two giant cranes holding a bridge collapse i...
7609,1,@aria_ahrary @TheTawniest The out of control...
7610,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. h...
7611,1,Police investigating after an e-bike collide...


## Задание 4 (0.5 балла)

Далее мы будем пока работать только с train частью.

1. Предобработайте данные (train часть) с помощью CountVectorizer.
2. Какого размера получилась матрица?


In [10]:
x_train, x_test, y_train, y_test = train_test_split(data.new, data.target, random_state=13)

In [11]:
vec = CountVectorizer()
bow = vec.fit_transform(x_train) 
bow

<5709x19267 sparse matrix of type '<class 'numpy.int64'>'
	with 92518 stored elements in Compressed Sparse Row format>

## Задание 5 (1 балл)

В предыдущем пункте у вас должна была получиться достаточно большая матрица.
Если вы взгляните на текст, то увидете, что там есть множество специальных символов, ссылок и прочего мусора.

Давайте также посмотрим на словарь, который получился в результате построения CountVectorizer, его можно найти в поле vocabulary_ инстанса этого класса. Давайте напишем функцию, которая печает ответы на следующие вопросы:

1. Найдите в этом словаре все слова, которые содержат цифры. Сколько таких слов нашлось?

2. Найдите все слова, которые содержат символы пунктуации. Сколько таких слов нашлось? 

3. Сколько хэштегов (токен начинается на #) и упоминаний (токен начинается на @) осталось в словаре?

In [12]:
vec.vocabulary_

{'sirens': 15395,
 'they': 16769,
 'them': 16736,
 'my': 11584,
 'dad': 4745,
 'said': 14776,
 'look': 10421,
 'thinner': 16785,
 'than': 16681,
 'usual': 17691,
 'but': 3330,
 'really': 14034,
 'im': 8648,
 'over': 12570,
 'here': 8037,
 'like': 10245,
 'http': 8352,
 'co': 4100,
 'bnwygx6luh': 2963,
 'burned': 3308,
 'oakland': 12188,
 'ca': 3383,
 'dog': 5407,
 'finds': 6626,
 'new': 11810,
 'home': 8217,
 'with': 18470,
 'young': 18985,
 'burn': 3306,
 'victim': 17884,
 'pqrjvgvgxg': 13328,
 'derailed': 5058,
 'toronto': 17037,
 'so': 15599,
 'derailed_benchmark': 5059,
 'is': 8999,
 'cool': 4378,
 'for': 6821,
 'paths': 12793,
 'wonder': 18529,
 'if': 8567,
 'can': 3471,
 'run': 14675,
 'it': 9033,
 'to': 16959,
 'find': 6625,
 'leaks': 10082,
 'in': 8710,
 'jobs': 9301,
 'given': 7364,
 'resque': 14290,
 'too': 17011,
 'fire': 6644,
 'canada': 3474,
 'on': 12375,
 'watsmxytva': 18191,
 'wounds': 18591,
 '11': 127,
 'puncture': 13610,
 'derail': 5057,
 'dayton': 4867,
 'oh': 12293

In [13]:
dict_keys = vec.vocabulary_.keys()
dict_keys

dict_keys(['sirens', 'they', 'them', 'my', 'dad', 'said', 'look', 'thinner', 'than', 'usual', 'but', 'really', 'im', 'over', 'here', 'like', 'http', 'co', 'bnwygx6luh', 'burned', 'oakland', 'ca', 'dog', 'finds', 'new', 'home', 'with', 'young', 'burn', 'victim', 'pqrjvgvgxg', 'derailed', 'toronto', 'so', 'derailed_benchmark', 'is', 'cool', 'for', 'paths', 'wonder', 'if', 'can', 'run', 'it', 'to', 'find', 'leaks', 'in', 'jobs', 'given', 'resque', 'too', 'fire', 'canada', 'on', 'watsmxytva', 'wounds', '11', 'puncture', 'derail', 'dayton', 'oh', 'realdonaldtrump', 'rushlimbaugh', 'its', 'proof', 'democrats', 'are', 'afraid', 'of', 'trump', 'clintons', 'trying', 'the', 'meddling', 'storm', 'nor', 'cal', 'rt', 'tonyhsieh', 'person', 'who', 'dances', 'you', 'rain', 'will', 'most', 'likely', 'walk', 'anonymous', 'deluge', 'brackley', 'beach', 'pe', 'trois', 'rivieres', 'about', 'one', 'hour', 'get', 'legionstrackandfield', 'pue5xnznqb', 'apocalypse', 'zonewolf123', 'liked', 'youtube', 'video',

In [27]:
word_and_number = []
word_with_number = []
word = [] 

def func(x: list) -> None:
    
    a = 0
    for i in x:
        if re.search('\w', i):           
            if re.search('\d', i):           
                if re.search('[A-Za-z]', i):
                    a += 1              
    
    print(a, '- слов с цифрами')
    
    n = 0
    for i in x:
        if re.search('\w', i): 
            if re.search('[.,!?:;]', i):
                n += 1                      
    
    print(n, '- слов содержащих пунктуацию')
    
    m = 0
    k = 0
    for i in x:
        if re.search('#.+', i):
            m += 1                      
        if re.search('@.+', i):
            k += 1
    
    print(m, '- хэштегов')
    print(k, '- упоминаний')
    
    

In [28]:
func(dict_keys)

3613 - слов с цифрами
0 - слов содержащих пунктуацию
0 - хэштегов
0 - упоминаний


## Задание 6 (0.5 балла)

Вспомним, что на семинаре по текстам мы узнали, что в nltk есть специальный токенизатор для текстов - TweetTokenizer. Попробуем применить CountVectorizer с этим токенизатором. Ответьте на все вопросы из предыдущего пункта для TweetTokenizer и сравните результаты.

In [29]:
#from nltk.tokenize import TweetTokenizer
# Чтобы узнать, какие параметры есть у этого токенайзера - используйте help(TweetTokenizer)
# Для того, чтобы передать токенайзер в CountVectorizer используйте параметр tokenizer

In [30]:
vec = CountVectorizer(tokenizer = TweetTokenizer().tokenize)
bow = vec.fit_transform(x_train)
bow

<5709x20582 sparse matrix of type '<class 'numpy.int64'>'
	with 101082 stored elements in Compressed Sparse Row format>

In [31]:
vec.vocabulary_

{'sirens': 17917,
 'they': 18976,
 '/': 1630,
 'them': 18958,
 'my': 15145,
 'dad': 6889,
 'said': 17394,
 'i': 13083,
 'look': 14303,
 'thinner': 18994,
 'than': 18937,
 'usual': 19630,
 'but': 5783,
 'really': 16830,
 'im': 13174,
 'over': 15776,
 'here': 9317,
 'like': 14170,
 'http://t.co/bnwygx6luh': 10234,
 'burned': 5762,
 'oakland': 15509,
 ',': 1597,
 'ca': 5803,
 'dog': 7403,
 'finds': 8315,
 'new': 15290,
 'home': 9450,
 'with': 20184,
 'young': 20433,
 'burn': 5761,
 'victim': 19742,
 'http://t.co/pqrjvgvgxg': 11704,
 'derailed': 7130,
 'toronto': 19172,
 'so': 18074,
 'derailed_benchmark': 7131,
 'is': 13462,
 'cool': 6624,
 'for': 8467,
 'paths': 15949,
 '.': 1622,
 'wonder': 20223,
 'if': 13131,
 'can': 5873,
 'run': 17332,
 'it': 13491,
 'to': 19110,
 'find': 8314,
 'leaks': 14055,
 'in': 13217,
 'jobs': 13616,
 'given': 8816,
 'resque': 17066,
 'too': 19151,
 '?': 2424,
 'fire': 8331,
 'canada': 5877,
 "i'm": 13086,
 'on': 15626,
 'http://t.co/watsmxytva': 12351,
 'wou

In [32]:
dict_keys = vec.vocabulary_.keys()

In [33]:
word_and_number = []
word_with_number = []
word = [] 

In [34]:
func(dict_keys)

3559 - слов с цифрами
3707 - слов содержащих пунктуацию
1583 - хэштегов
1810 - упоминаний


In [399]:
nltk.corpus.stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Задание 7 (2 балла)

В scikit-learn мы можем оценивать процесс подсчета матрицы через CountVectorizer. У CountVectorizer, как и у других наследников \_VectorizerMixin, есть аргумент tokenizer и preprocessor. preprocessor применится в самом начале к каждой строке вашего датасета, tokenizer же должен принять строку и вернуть токены.
Давайте напишем кастомный токенайзер, которые сделает все, что нам нужно: 

0. Приведет все буквы к нижнему регистру
1. Разобьет текст на токены с помощью TweetTokenizer из пакета nltk
2. Удалит все токены содержащие не латинские буквы, кроме смайликов (будем считать ими токены содержащие только пунктуацию и, как минимум, одну скобочку) и хэштегов, которые после начальной # содержат только латинские буквы.
3. Удалит все токены, которые перечислены в nltk.corpus.stopwords.words('english')
4. Проведет стемминг с помощью SnowballStemmer

Продемонстрируйте работу вашей функции на первых десяти текстах в обучающей выборке.

In [380]:
def top_tokenizer(text: str) -> list:
    stop_words = nltk.corpus.stopwords.words('english')
    text = text.lower()
    #print(text)
    tokenized_text = TweetTokenizer().tokenize(text)
    #print(tokenized_text)
    tokenized_text2 = []
    tokenized_text3 = []
    tokenized_text4 = []
    
    for i in tokenized_text:
        if not re.search('[^A-Za-z0-9.,>?<;:|@$#%()-/]', i): # ищет не латинские буквы
            tokenized_text2.append(i)
    for i in tokenized_text2:
        if not re.search('http://.+', i): # убрал все ссылки
            tokenized_text3.append(i)
    for i in tokenized_text3:             # стоп слова
        if i not in stop_words:
            tokenized_text4.append(i)
    
    stemmer = SnowballStemmer('english')
    stemmed_text = [stemmer.stem(w) for w in tokenized_text4]
    
            
    return tokenized_text4

In [398]:
top_tokenizer('Ghoul Привет. нет, day21 notрашн крот крот :) @fartuk #hashtag + @прит ©   ¢ #аоолыв sdыва 123 213ррло 13sddf  http://t.co/bnwygx6luh \x89 :-) ? ! _ #directioners () * & ^ 23 | = i - å')

['ghoul',
 '.',
 ',',
 'day',
 '21',
 ':)',
 '@fartuk',
 '#hashtag',
 '+',
 '123',
 '13sddf',
 ':-)',
 '?',
 '#directioners',
 '(',
 ')',
 '*',
 '23',
 '|',
 '-']

In [392]:
for i in x_train.iloc[0:10]:
    print(top_tokenizer(i))

['sirens', '/', 'dad', 'said', 'look', 'thinner', 'usual', 'really', 'im', 'like']
['burned', 'oakland', ',', 'ca', 'burned', 'dog', 'finds', 'new', 'home', 'young', 'burn', 'victim']
['derailed', 'toronto', 'cool', 'paths', '.', 'wonder', 'run', 'find', 'leaks', 'jobs', 'given', 'resque', '?']
['fire', 'canada', 'fire', '.']
['wounds', '11', 'puncture', 'wounds']
['derail', 'dayton', ',', 'oh', '@realdonaldtrump', '@rushlimbaugh', 'proof', 'democrats', 'afraid', 'trump', 'clintons', 'trying', 'derail', 'trump', 'w', 'meddling']
['storm', '*', 'cal', 'rt', '@tonyhsieh', ':', 'person', 'dances', 'rain', 'likely', 'walk', 'storm', '.', '-', 'anonymous']
['deluge', 'brackley', 'beach', ',', 'pe', ',', 'canada', 'deluge', 'trois-rivieres', '.', 'one', 'hour', 'get', '#legionstrackandfield']
['apocalypse', 'rt', ':', 'zonewolf', '123', ':', 'liked', 'youtube', 'video', 'minecraft', ':', 'night', 'lucky', 'block', 'mod', '(', 'bob', 'apocalypse', 'wither', '2.0', ')', 'mo']
['accident', 'uk'

## Задание 8 (1 балл)

1. Примените CountVectorizer с реализованным выше токенизатором к обучающим и тестовым выборкам.
2. Обучите LogisticRegression на полученных признаках.
3. Посчитайте метрику f1-score на тестовых данных.

In [383]:
vec = CountVectorizer(tokenizer = top_tokenizer)

In [384]:
bow = vec.fit_transform(x_train)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

bow

<5709x16506 sparse matrix of type '<class 'numpy.float64'>'
	with 68891 stored elements in Compressed Sparse Row format>

In [385]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(f1_score(y_test, pred))

0.722112211221122


## Задание 9 (1 балл)

1. Повторите 8 задание, но с tf-idf векторизатором. Как изменилось качество?
2. Мы можем еще сильнее уменьшить размер нашей матрицы, если отбросим значения df близкие к единице. Скорее всего такие слова не несут много информации о категории, так как встречаются достаточно часто. Ограничьте максимальный df в параметрах TfIdfVectorizer, поставьте верхнюю границу равную 0.9. Как изменился размер матрицы, как изменилось качество?
3. Также мы можем уменьшить размер матрицы, удаляя слова со слишком маленьким df. Удалось ли добиться улучшения качества? 

In [393]:
vec = TfidfVectorizer(tokenizer = top_tokenizer)
vec_train = vec.fit_transform(x_train)
vec_test = vec.transform(x_test)

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

vec_train

<5709x16506 sparse matrix of type '<class 'numpy.float64'>'
	with 68891 stored elements in Compressed Sparse Row format>

In [394]:
clf = LogisticRegression(random_state=42)
clf.fit(vec_train, y_train)
pred = clf.predict(vec_test)
print(f1_score(y_test, pred))

0.7240932642487047


In [395]:
vec = TfidfVectorizer(tokenizer = top_tokenizer, min_df = 10e-4) 
vec_train = vec.fit_transform(x_train)
vec_test = vec.transform(x_test)

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

vec_train

<5709x1998 sparse matrix of type '<class 'numpy.float64'>'
	with 47164 stored elements in Compressed Sparse Row format>

In [396]:
clf = LogisticRegression(random_state=42)
clf.fit(vec_train, y_train)
pred_tfidf = clf.predict(vec_test)
print(f1_score(y_test, pred_tfidf))

0.723348593852191


## Задание 10 (1 балл)

Еще один популяпный трюк, который позволит уменьшить количество признаков называется hashing trick. Его суть в том, что мы случайно группируем признаки ииии  ..... складываем их! А потом удаляем исходные признаки. В итоге все наши признаки это просто суммы исходных. Звучит странно, но это отлично работает. Давайте проверим этот трюк в нашем сеттинге.
Также при таком подходе вам не нужно хранить словарь token->index, что тоже иногда полезно.

1. Повторите задание 8 с HashingVectorizer, укажите количество фичей равное 5000.
2. Какой из подходов показал самый высокий результат?

In [232]:
vec = HashingVectorizer(n_features = 5000, tokenizer = top_tokenizer)
vec_train = vec.fit_transform(x_train)
vec_test = vec.transform(x_test)

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

vec_train

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<5709x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 71638 stored elements in Compressed Sparse Row format>

In [233]:
clf.fit(vec_train, y_train)
pred = clf.predict(vec_test)
print(f1_score(y_test, pred))

0.6829268292682926


## Задание 11 (1 балл)

В этом задании нужно добиться f1 меры хотя бы в 0.75 на тестовых данных.